In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler #標準化のコード

# データの読み込み
# ファイル名を実際の訓練・テストデータファイル名に置き換えてください
train_data_LR = pd.read_csv('selected_train_set.csv')
test_data_LR = pd.read_csv('selected_test_set.csv')
#index_col=['id']

# 特徴量（X）と目標変数（y）の設定
#11/1のミーティングで決まった項目も抜いて機械学習

X_train_LR = train_data_LR.drop(columns=['PassengerId','Survived'])  # 'target_column'を目標変数の列名に置き換えてください
y_train_LR = train_data_LR['Survived']
X_test_LR = test_data_LR.drop(columns=['PassengerId','Survived'])
y_test_LR = test_data_LR['Survived']

# 特徴量の数を確認
num_features_LR = X_train_LR.shape[1]
print("特徴量の数:", num_features_LR)

# 標準化
scaler = StandardScaler()
X_train_standard_LR = scaler.fit_transform(X_train_LR)
X_test_standard_LR = scaler.transform(X_test_LR)
# 10回ループのためのリストを初期化
accuracies_LR = []  # 精度を格納するリスト
# 重要度を格納するリスト
importances_list_LR = []

predicted_probabilities_LR = []  # 予測確率を格納するリスト
random_state = 42
# 10回ループでモデルを作成
for i in range(10):
    # sagaソルバーとElastic Netペナルティを使用したロジスティック回帰モデルを定義
    model_LR = LogisticRegression(
    solver='saga',           # 小さなデータセットに最適なソルバーで正則化対応
    penalty='elasticnet',    # Elastic Netで特徴選択と正則化を両立
    l1_ratio=0.5,            # L1とL2のバランス（0.5は出発点として適切）
    C=1.0,                   # 正則化の強度（調整やチューニングも可能）
    max_iter=10000,            # 収束を確実にするために最大イテレーションを増加   #5000だと精度0.50、
    random_state=random_state  # Set random state for reproducibility
)
    # ロジスティック回帰モデルの作成
    #model = LogisticRegression(max_iter=100, random_state=i)  # max_iterで反復回数を調整 #100で0.50,200で0.44
    #model = LogisticRegression(solver='liblinear')#0.44だった

    # モデルの訓練
    model_LR.fit(X_train_standard_LR, y_train_LR)

# 回帰係数を取得（11/1）
    coefficients_LR = model_LR.coef_[0]  # 係数は配列として返されるので、最初の要素を取得
    
    # テストデータでの予測
    y_pred_LR = model_LR.predict(X_test_standard_LR)

    # 特徴量の重要度を取得し、リストに追加 11/1
    importances_list_LR.append(np.abs(coefficients_LR))  # 絶対値を取って追加

    # テストデータで予測確率を取得し、リストに追加
    y_scores_LR = model_LR.predict_proba(X_test_standard_LR)[:, 1]
    predicted_probabilities_LR.append(y_scores_LR)
    
    # モデルの精度を計算 このモデルの適応度みたいなもの
    accuracy_LR = accuracy_score(y_test_LR, y_pred_LR)
    accuracies_LR.append(accuracy_LR)

# 平均精度を計算
average_accuracy_LR = np.mean(accuracies_LR)
print(f'Average Accuracy over 10 runs: {average_accuracy_LR:.2f}')

from sklearn.model_selection import cross_val_score   #一般化敵には、max_iterは10000の方がよい。収束している
accuracies_LR = cross_val_score(model_LR, X_train_standard_LR, y_train_LR, cv=10)
print(f'Average Accuracy over 10 folds: {np.mean(accuracies_LR):.2f}')

# 特徴量の重要度の平均を計算
mean_importances_LR = np.mean(importances_list_LR, axis=0)
# 特徴量の名前と重要度の平均をデータフレームにまとめる
feature_importance_df_LR = pd.DataFrame({
    'Feature': X_train_LR.columns,
    'Importance': mean_importances_LR
})

# 重要度でソート
feature_importance_df_LR = feature_importance_df_LR.sort_values(by='Importance', ascending=False)
# 上位13項目を抽出
top_13_importance_df_LR = feature_importance_df_LR.head(13)

# グラフの作成
plt.figure(figsize=(20, 16))
plt.barh(top_13_importance_df_LR['Feature'], top_13_importance_df_LR['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.title('Top 13 Feature Importances (Random Forest)')
plt.gca().invert_yaxis()
plt.show()

from sklearn.metrics import confusion_matrix, roc_curve, auc

y_scores_LR = model_LR.predict_proba(X_test_standard_LR)[:, 1]  # クラス1の確率を取得
#print(y_scores)

# ROC計算用のしきい値を決定（スコアの一意な値を使用）
thresholds_LR = np.sort(np.unique(y_scores_LR))
# 感度（TPR）と特異度を計算
tpr_list_LR = []
specificity_list_LR = []

for threshold in thresholds_LR:
    # スコアをしきい値で2値化
    y_pred_LR = (y_scores_LR >= threshold).astype(int)
    
    # 混同行列を取得
    tn_LR, fp_LR, fn_LR, tp_LR = confusion_matrix(y_test_LR, y_pred_LR).ravel()
    
    # 感度と特異度を計算
    tpr_LR = tp_LR / (tp_LR + fn_LR) if (tp_LR + fn_LR) > 0 else 0  # 感度
    specificity_LR = tn_LR / (tn_LR + fp_LR) if (tn_LR + fp_LR) > 0 else 0  # 特異度
    
    # リストに保存
    tpr_list_LR.append(tpr_LR)
    specificity_list_LR.append(specificity_LR)

# AUCの計算
roc_auc_LR = auc(np.array(specificity_list_LR), tpr_list_LR)

# ROC曲線を描画
plt.figure(figsize=(8, 6))
plt.plot( np.array(specificity_list_LR), tpr_list_LR, label=f"ROC Curve (AUC = {roc_auc_LR:.2f})", color="blue")
plt.plot([1, 0], [0, 1], linestyle="--", color="red", label="Random Guess")

# グラフの装飾
plt.xlim([1.0, 0.0])  # X軸を1から0に反転
plt.ylim([0.0, 1.0])
plt.xlabel("Specificity ")
plt.ylabel("Sensitivity")
plt.title("ROC Curve with TP, FP, TN, FN")
plt.legend(loc="lower left")
plt.grid()
plt.show()
from sklearn.metrics import confusion_matrix
# 混同行列の計算
tn_LR, fp_LR, fn_LR, tp_LR = confusion_matrix(y_test_LR, y_pred_LR).ravel()

# 感度と特異度の計算
sensitivity_LR = tp_LR / (tp_LR + fn_LR)  # 真陽性率
specificity_LR = tn_LR / (tn_LR + fp_LR)  # 真陰性率

print(f"Sensitivity: {sensitivity_LR:.4f}")
print(f"Specificity: {specificity_LR:.4f}")

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

# ファイル名を実際の訓練・テストデータファイル名に置き換えてください
train_data_xgb = pd.read_csv('selected_train_set.csv')
test_data_xgb = pd.read_csv('selected_test_set.csv')
index_col=['id']
# 特徴量（X）と目標変数（y）の設定
X_train_xgb = train_data_xgb.drop(columns=['PassengerId','Survived'])  # 'target_column'を目標変数の列名に置き換えてください
y_train_xgb = train_data_xgb['Survived']
X_test_xgb = test_data_xgb.drop(columns=['PassengerId','Survived'])
y_test_xgb = test_data_xgb['Survived']

# クロスバリデーションで得た最適なブーストラウンド数
optimal_boost_rounds = 28  # ここで得た値

# クラスの不均衡を考慮した重みの計算
ratio = y_train_xgb.value_counts()[0] / y_train_xgb.value_counts()[1]  # 負例/正例の比率

# 10回ループの準備
accuracies_xgb = []  # 精度を格納するリスト


for i in range(10):
    # 訓練-検証の分割
    X_train_split_xgb, X_val_split_xgb, y_train_split_xgb, y_val_split_xgb = train_test_split(X_train_xgb, y_train_xgb, test_size=0.2, random_state=42)

#以下に新しく提案されたコードを各
    # DMatrixに変換
    dtrain_xgb = xgb.DMatrix(X_train_split_xgb, label=y_train_split_xgb)
    dval_xgb = xgb.DMatrix(X_val_split_xgb, label=y_val_split_xgb)
    dtest_xgb = xgb.DMatrix(X_test_xgb)

    params = {
        'objective':'binary:logistic',
        'eval_metric':'logloss',
        'random_state':42,
        'learning_rate':0.1,
        'max_depth':5,
        'subsample':1.0,
        'colsample_bytree':1.0,
        'gamma':0.1
}
# アーリーストッピングでモデルを訓練
    evals_xgb = [(dtrain_xgb, 'train'), (dval_xgb, 'eval')]
    model_xgb = xgb.train(params, dtrain_xgb, num_boost_round=optimal_boost_rounds, early_stopping_rounds=10, evals=evals_xgb, verbose_eval=False)
    #num_boost_roundを100(0.51)から200に、learn_rateを0.1から0.05に（過学習抑制）、max_depthを3からまずは4に変更
    # 予測
    y_test_pred_xgb = model_xgb.predict(dtest_xgb)
    y_test_pred_xgb = [1 if pred > 0.5 else 0 for pred in y_test_pred_xgb]  # 二値分類として閾値0.5で予測値を変換
    accuracy_xgb = accuracy_score(y_test_xgb, y_test_pred_xgb)
    accuracies_xgb.append(accuracy_xgb)

# 平均精度を計算
average_accuracy_xgb = np.mean(accuracies_xgb)
print(f'Average Accuracy over 10 runs: {average_accuracy_xgb:.2f}')

import matplotlib.pyplot as plt
# 特徴量の重要度を取得 (weightとgain)
importance_weight_xgb = model_xgb.get_score(importance_type='weight')

# 全ての特徴量を取得してデータフレームにまとめる
all_features_xgb = X_train_split_xgb.columns
importance_df_xgb = pd.DataFrame({
    'Feature': all_features_xgb,
    'Weight': [importance_weight_xgb.get(feature, 0) for feature in all_features_xgb],  # Weightがない場合は0
})


# データフレームにまとめて表示
importance_df_xgb = pd.DataFrame({
    'Feature': list(importance_weight_xgb.keys()),
    'Weight': list(importance_weight_xgb.values()),
   
})

# Weightで上位13項目をソート
top_13_weight_df_xgb = importance_df_xgb.sort_values(by='Weight', ascending=False).head(13)


# 結果の表示
print("Top 13 Feature Importances by Weight:\n", top_13_weight_df_xgb)


# グラフの作成
plt.figure(figsize=(10,8))

# Weightのプロット
plt.subplot(1, 2, 2)
plt.barh(top_13_weight_df_xgb['Feature'], top_13_weight_df_xgb['Weight'], color='lightgreen')
plt.xlabel('Weight')
plt.title('Top 13 Feature Importances by Weight')
plt.gca().invert_yaxis()

plt.tight_layout()
plt.show()

from sklearn.metrics import roc_curve, auc,confusion_matrix

# テストデータをDMatrix形式に変換
dtest = xgb.DMatrix(X_test_xgb)
# テストデータでの予測確率を取得
y_scores_xgb = model_xgb.predict(dtest_xgb)

# ROC計算用のしきい値を決定（スコアの一意な値を使用）
thresholds_xgb = np.sort(np.unique(y_scores_xgb))

# 感度（TPR）と特異度を計算
tpr_list_xgb = []
specificity_list_xgb = []

for threshold in thresholds_xgb:
    # スコアをしきい値で2値化
    y_pred_xgb = (y_scores_xgb >= threshold).astype(int)
    
    # 混同行列を取得
    tn_xgb, fp_xgb, fn_xgb, tp_xgb = confusion_matrix(y_test_xgb, y_pred_xgb).ravel()
    
    # 感度と特異度を計算
    tpr_xgb = tp_xgb / (tp_xgb + fn_xgb) if (tp_xgb + fn_xgb) > 0 else 0  # 感度
    specificity_xgb = tn_xgb / (tn_xgb + fp_xgb) if (tn_xgb + fp_xgb) > 0 else 0  # 特異度
    
    # リストに保存
    tpr_list_xgb.append(tpr_xgb)
    specificity_list_xgb.append(specificity_xgb)

# AUCの計算
roc_auc_xgb = auc(1 - np.array(specificity_list_xgb), tpr_list_xgb)

# ROC曲線を描画
plt.figure(figsize=(8, 6))
plt.plot(np.array(specificity_list_xgb), tpr_list_xgb, label=f"ROC Curve (AUC = {roc_auc_xgb:.2f})", color="blue")
plt.plot([1, 0], [0, 1], color="gray", linestyle="--")  # 45度線
plt.xlim([1.0, 0.0])  # X軸を1から0に設定
plt.ylim([0.0, 1.0])  # Y軸を0から1に設定
plt.xlabel("Specificity")
plt.ylabel("Sensitivity")
plt.title("ROC Curve (Specificity vs Sensitivity)")
plt.legend(loc="lower left")
plt.grid()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


# ファイル名を実際の訓練・テストデータファイル名に置き換えてください
train_data_rf = pd.read_csv('selected_train_set.csv')
test_data_rf = pd.read_csv('selected_test_set.csv')
index_col=['id']
# 特徴量（X）と目標変数（y）の設定
X_train_rf = train_data_rf.drop(columns=['PassengerId','Survived'])  # 'target_column'を目標変数の列名に置き換えてください
y_train_rf = train_data_rf['Survived']
X_test_rf = test_data_rf.drop(columns=['PassengerId','Survived'])
y_test_rf = test_data_rf['Survived']

# スケーリングとモデルのパイプライン設定
pipeline_rf = Pipeline([
    #('scaler', StandardScaler()),RFのみの時は、下のコードのみでよい
    ('model', RandomForestClassifier(n_estimators=200, random_state=42, max_depth=5, min_samples_split=5))
])


# 10回ループするためのリストを初期化
accuracies_rf = []  # 精度を格納するリスト
importances_list_rf = []  # 特徴量の重要度を格納するリスト
# 10回ループでモデルを作成
for i in range(10):
    # ランダムフォレストモデルの作成
    model_rf = RandomForestClassifier(n_estimators=200, random_state=i,max_depth=5, min_samples_split=5)
    
    # モデルの訓練
    model_rf.fit(X_train_rf, y_train_rf)

    # テストデータでの予測
    y_pred_rf = model_rf.predict(X_test_rf)
    
    # モデルの精度を計算
    accuracy_rf = accuracy_score(y_test_rf, y_pred_rf)
    accuracies_rf.append(accuracy_rf)
    
    # 特徴量の重要度を取得してリストに追加
    importances_rf = model_rf.feature_importances_
    importances_list_rf.append(importances_rf)

# 平均精度を計算
average_accuracy_rf = np.mean(accuracies_rf)
print(f'Average Accuracy over 10 runs: {average_accuracy_rf:.2f}')

# 特徴量の重要度の平均を計算
mean_importances_rf = np.mean(importances_list_rf, axis=0)
# 特徴量の名前と重要度の平均をデータフレームにまとめる
feature_importance_df_rf = pd.DataFrame({
    'Feature': X_train_rf.columns,
    'Importance': mean_importances_rf
})

# 重要度でソートし、上位13項目を抽出
top_13_features_rf = feature_importance_df_rf.sort_values(by='Importance', ascending=False).head(13)

# プロット
plt.barh(top_13_features_rf['Feature'], top_13_features_rf['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.title('Average Feature Importance over 10 runs (Random Forest)')
plt.gca().invert_yaxis()  # 特徴量を重要度が高い順に表示
plt.show()

from sklearn.metrics import roc_curve, auc,confusion_matrix
# テストデータで予測確率を取得
y_scores_rf = model_rf.predict_proba(X_test_rf)[:, 1]

# ROC計算用のしきい値を決定（スコアの一意な値を使用）
thresholds_rf = np.sort(np.unique(y_scores_rf))

# 感度（TPR）と特異度を計算
tpr_list_rf = []
specificity_list_rf = []

for threshold in thresholds_rf:
    # スコアをしきい値で2値化
    y_pred_rf = (y_scores_rf >= threshold).astype(int)
    
    # 混同行列を取得
    tn_rf, fp_rf, fn_rf, tp_rf = confusion_matrix(y_test_rf, y_pred_rf).ravel()
    
    # 感度と特異度を計算
    tpr_rf = tp_rf / (tp_rf+ fn_rf) if (tp_rf + fn_rf) > 0 else 0  # 感度
    specificity_rf = tn_rf / (tn_rf + fp_rf) if (tn_rf + fp_rf) > 0 else 0  # 特異度
    
    # リストに保存
    tpr_list_rf.append(tpr_rf)
    specificity_list_rf.append(specificity_rf)

# AUCの計算
roc_auc_rf = auc(np.array(specificity_list_rf), tpr_list_rf)

# ROC曲線を描画
plt.figure(figsize=(8, 6))
plt.plot( np.array(specificity_list_rf), tpr_list_rf, label=f"ROC Curve (AUC = {roc_auc_rf:.2f})", color="blue")
plt.plot([1, 0], [0, 1], color="red", linestyle="--")  # 45度線
plt.xlim([1.0, 0.0])  # X軸を1から0に設定
plt.xlabel("Specificity ")
plt.ylabel("Sensitivity ")
plt.title("ROC Curve (Specificity vs Sensitivity)")
plt.legend(loc="lower left")
plt.grid()
plt.show()

In [ ]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
import numpy as np
import matplotlib.pyplot as plt

# データの読み込み
train_data_nn = pd.read_csv('selected_train_set.csv')
test_data_nn = pd.read_csv('selected_test_set.csv')

# 特徴量と目的変数の分離
X_train_nn = train_data_nn.drop(columns=['PassengerId','Survived'])
y_train_nn = train_data_nn['Survived']
X_test_nn = test_data_nn.drop(columns=['PassengerId','Survived'])
y_test_nn = test_data_nn['Survived']

# データの標準化
scaler_nn = StandardScaler()
X_train_scaled_nn = scaler_nn.fit_transform(X_train_nn)
X_test_scaled_nn = scaler_nn.transform(X_test_nn)

# 標準化されたデータをデータフレームに戻し、元のカラム名を保持
X_train_scaled_df_nn = pd.DataFrame(X_train_scaled_nn, columns=X_train_nn.columns, index=X_train_nn.index)
X_test_scaled_df_nn = pd.DataFrame(X_test_scaled_nn, columns=X_test_nn.columns, index=X_test_nn.index)
# ニューラルネットワークのパラメータグリッドの設定
param_grid_nn = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],  # 隠れ層の構成
    'activation': ['relu', 'tanh'],                   # 活性化関数
    'learning_rate_init': [0.001, 0.01],              # 学習率
    'max_iter': [200, 500]                            # 最大エポック数
}

# 重要度を格納するリスト
importances_list_nn = []
# 予測確率を格納するリスト
predicted_probabilities_nn = []
# 精度を格納するリスト
accuracy_scores_nn = []

# モデルのトレーニングと評価を10回繰り返す
for i in range(10):
    # ニューラルネットワークモデルの初期化
    model_nn = MLPClassifier(random_state=42)


    # GridSearchCVの設定
    grid_search_nn = GridSearchCV(estimator=model_nn, param_grid=param_grid_nn, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# 訓練データでモデルをトレーニングし、最適なハイパーパラメータを取得
    grid_search_nn.fit(X_train_scaled_df_nn, y_train_nn)
    best_model_nn = grid_search_nn.best_estimator_

    
    # 訓練データでモデルをトレーニング
    #model_nn.fit(X_train_scaled_df_nn, y_train_nn)

    # 特徴量の重要度を取得し、リストに追加
    #importances_list_nn.append(best_model_nn.feature_importances_)

    # テストデータで予測確率を取得し、リストに追加
    y_scores_nn = best_model_nn.predict_proba(X_test_scaled_df_nn)[:, 1]
    predicted_probabilities_nn.append(y_scores_nn)

    # テストデータで予測を行う
    y_pred_nn = best_model_nn.predict(X_test_scaled_df_nn)
# Permutation Importanceの計算
    result_nn = permutation_importance(best_model_nn, X_test_scaled_df_nn, y_test_nn, n_repeats=10, random_state=42, n_jobs=-1)
    importances_list_nn.append(result_nn.importances_mean)

    # テストデータで予測確率を取得し、リストに追加
    y_scores_nn = best_model_nn.predict_proba(X_test_scaled_df_nn)[:, 1]
    predicted_probabilities_nn.append(y_scores_nn)

    # テストデータで予測を行う
    y_pred_nn = best_model_nn.predict(X_test_scaled_df_nn)
    
# 精度を計算してリストに追加
    accuracy_nn = accuracy_score(y_test_nn, y_pred_nn)
    accuracy_scores_nn.append(accuracy_nn)

# 10回の平均精度を計算
average_accuracy_nn = np.mean(accuracy_scores_nn)

# 平均精度を出力
print(f"\n平均精度 (10回): {average_accuracy_nn:.4f}")


# 特徴量の重要度の平均を計算
mean_importances_nn = np.mean(importances_list_nn, axis=0)

# 特徴量の名前と重要度の平均をデータフレームにまとめる
feature_importance_df_nn = pd.DataFrame({
    'Feature': X_train_nn.columns,
    'Importance': mean_importances_nn
})

# 重要度でソート
feature_importance_df_nn = feature_importance_df_nn.sort_values(by='Importance', ascending=False)
# 上位13項目を抽出
top_13_importance_df_nn = feature_importance_df_nn.head(13)

# グラフの作成
plt.figure(figsize=(10, 8))
plt.barh(top_13_importance_df_nn['Feature'], top_13_importance_df_nn['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.title('Top 13 Feature Importances (Random Forest)')
plt.gca().invert_yaxis()
plt.show()

# 平均予測確率を計算
mean_predicted_probabilities_nn = np.mean(predicted_probabilities_nn, axis=0)

from sklearn.metrics import confusion_matrix, roc_curve, auc


# テストデータで予測確率を取得
y_scores_nn = best_model_nn.predict_proba(X_test_scaled_df_nn)[:, 1]

# ROC計算用のしきい値を決定（スコアの一意な値を使用）
thresholds_nn = np.sort(np.unique(y_scores_nn))

# 感度（TPR）と特異度を計算
tpr_list_nn = []
specificity_list_nn = []

for threshold in thresholds_nn:
    # スコアをしきい値で2値化
    y_pred_nn = (y_scores_nn >= threshold).astype(int)
    
    # 混同行列を取得
    tn_nn, fp_nn, fn_nn, tp_nn = confusion_matrix(y_test_nn, y_pred_nn).ravel()

    # 感度と特異度を計算
    tpr_nn = tp_nn / (tp_nn+ fn_nn) if (tp_nn + fn_nn) > 0 else 0  # 感度
    specificity_nn = tn_nn / (tn_nn + fp_nn) if (tn_nn + fp_nn) > 0 else 0  # 特異度
    
    # リストに保存
    tpr_list_nn.append(tpr_nn)
    specificity_list_nn.append(specificity_nn)

# AUCの計算
roc_auc_nn = auc(np.array(specificity_list_nn), tpr_list_nn)

# ROC曲線を描画
plt.figure(figsize=(8, 6))
plt.plot( np.array(specificity_list_nn), tpr_list_nn, label=f"ROC Curve (AUC = {roc_auc_nn:.2f})", color="blue")

plt.plot([1, 0], [0, 1], linestyle="--", color="red", label="Random Guess")

# グラフの装飾
plt.xlim([1.0, 0.0])  # X軸を1から0に反転
plt.ylim([0.0, 1.0])
plt.xlabel("Specificity ")
plt.ylabel("Sensitivity")
plt.title("ROC Curve with TP, FP, TN, FN")
plt.legend(loc="lower left")
plt.grid()
plt.show()

from sklearn.metrics import confusion_matrix
# 混同行列の計算
tn_nn, fp_nn, fn_nn, tp_nn = confusion_matrix(y_test_nn, y_pred_nn).ravel()

# 感度と特異度の計算
sensitivity_nn = tp_nn / (tp_nn + fn_nn)  # 真陽性率
specificity_nn = tn_nn / (tn_nn + fp_nn)  # 真陰性率

print(f"Sensitivity: {sensitivity_nn:.4f}")
print(f"Specificity: {specificity_nn:.4f}")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# データの読み込み
train_data_senkei = pd.read_csv('selected_train_set.csv')
test_data_senkei = pd.read_csv('selected_test_set.csv')

# 特徴量（X）と目標変数（y）の設定
X_train_senkei = train_data_senkei.drop(columns=['PassengerId','Survived'])
y_train_senkei = train_data_senkei['Survived']
X_test_senkei = test_data_senkei.drop(columns=['PassengerId','Survived'])
y_test_senkei = test_data_senkei['Survived']

# データの標準化
scaler_senkei = StandardScaler()
X_train_senkei = scaler_senkei.fit_transform(X_train_senkei)
X_test_senkei = scaler_senkei.transform(X_test_senkei)

param_grid_linear_senkei = {'C': [0.01, 0.1, 1, 10, 100]}
grid_search_linear_senkei = GridSearchCV(SVC(kernel='linear', probability=True, random_state=42), param_grid_linear_senkei, cv=10, scoring='accuracy')
grid_search_linear_senkei.fit(X_train_senkei, y_train_senkei)
best_model_senkei = grid_search_linear_senkei.best_estimator_

# 線形カーネルのSVMモデルを10回評価 class_weight='balanced'不均衡データに対応するコード0.5→あまり変わらず
linear_accuracies_senkei = []

for i in range(10):
    svm_linear_senkei = SVC(kernel='linear', probability=True, random_state=42,class_weight='balanced')
    svm_linear_senkei.fit(X_train_senkei, y_train_senkei)
    y_pred_senkei = best_model_senkei.predict(X_test_senkei)
    #y_pred_linear_senkei = svm_linear_senkei.predict(X_test_svm_senkei)
    accuracy_senkei = accuracy_score(y_test_senkei, y_pred_senkei)
    linear_accuracies_senkei.append(accuracy_senkei)

# 線形カーネルSVMの平均精度の表示
print("線形カーネルSVMの10回の平均精度:", np.mean(linear_accuracies_senkei))
print("線形カーネルSVMの詳細な評価結果:\n", classification_report(y_test_senkei, y_pred_senkei))

import matplotlib.pyplot as plt
# 特徴量の重要度の計算
importances_senkei = best_model_senkei.coef_[0]  # 線形SVMの係数が重要度として使える
feature_importance_df_senkei = pd.DataFrame({
    'Feature': train_data_senkei.drop(columns=['PassengerId','Survived']).columns,
    'Importance': np.abs(importances_senkei)  # 絶対値を取って重要度を見やすくする
})

# 重要度でソートして上位13項目を抽出
feature_importance_df_senkei = feature_importance_df_senkei.sort_values(by='Importance', ascending=False)
top_13_importance_df_senkei = feature_importance_df_senkei.head(13)

# グラフの作成
plt.figure(figsize=(10, 8))
plt.barh(top_13_importance_df_senkei['Feature'], top_13_importance_df_senkei['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.title('Top 13 Feature Importances (SVM Linear Kernel)')
plt.gca().invert_yaxis()
plt.show()

# モデルの精度評価
y_pred_linear_senkei = best_model_senkei.predict(X_test_senkei)
accuracy_linear_senkei = accuracy_score(y_test_senkei, y_pred_senkei)
print("線形カーネルSVMのテスト精度:", accuracy_senkei)
print("線形カーネルSVMの詳細な評価結果:\n", classification_report(y_test_senkei, y_pred_senkei))

from sklearn.metrics import roc_curve, auc,confusion_matrix

# テストデータで予測確率を取得
y_scores_senkei = best_model_senkei.predict_proba(X_test_senkei)[:, 1]

# ROC計算用のしきい値を決定（スコアの一意な値を使用）
thresholds_senkei = np.sort(np.unique(y_scores_senkei))

# 感度（TPR）と特異度を計算
tpr_list_senkei = []
specificity_list_senkei = []

for threshold in thresholds_senkei:
    # スコアをしきい値で2値化
    y_pred_senkei = (y_scores_senkei >= threshold).astype(int)
    
    # 混同行列を取得
    tn_senkei, fp_senkei, fn_senkei, tp_senkei = confusion_matrix(y_test_senkei, y_pred_senkei).ravel()
    
    # 感度と特異度を計算
    tpr_senkei = tp_senkei / (tp_senkei+ fn_senkei) if (tp_senkei + fn_senkei) > 0 else 0  # 感度
    specificity_senkei = tn_senkei / (tn_senkei + fp_senkei) if (tn_senkei + fp_senkei) > 0 else 0  # 特異度
    
    # リストに保存
    tpr_list_senkei.append(tpr_senkei)
    specificity_list_senkei.append(specificity_senkei)

# AUCの計算
roc_auc_senkei = auc(np.array(specificity_list_senkei), tpr_list_senkei)

# ROC曲線を描画
plt.figure(figsize=(8, 6))
plt.plot( np.array(specificity_list_senkei), tpr_list_senkei, label=f"ROC Curve (AUC = {roc_auc_senkei:.2f})", color="blue")

plt.plot([1, 0], [0, 1], color="red", linestyle="--")  # 45度線
plt.xlim([1.0, 0.0])  # X軸を1から0に設定
plt.xlabel("Specificity")
plt.ylabel("Sensitivity")
plt.title("ROC Curve (Specificity vs Sensitivity)")
plt.legend(loc="lower left")
plt.grid()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc,confusion_matrix
import matplotlib.pyplot as plt

# データの読み込み
train_data_sig = pd.read_csv('selected_train_set.csv')
test_data_sig = pd.read_csv('selected_test_set.csv')

# 特徴量（X）と目標変数（y）の設定
X_train_sig = train_data_sig.drop(columns=['PassengerId','Survived'])
y_train_sig = train_data_sig['Survived']
X_test_sig = test_data_sig.drop(columns=['PassengerId','Survived'])
y_test_sig = test_data_sig['Survived']

# データの標準化
scaler_sig = StandardScaler()
X_train_sig_scaler = scaler_sig.fit_transform(X_train_sig)
X_test_sig_scaler = scaler_sig.transform(X_test_sig)


#C:0.001,0.01,0.1,1,10,100,gamma:0.0001,0.001,0.01,0.1,1で0.5
param_grid_sig = {
    'C': [0.001, 0.01, 0.1, 1, 10,100],
    'gamma': [0.0001, 0.001, 0.01, 0.1,1]
}
# RBF→sigカーネルのSVMでハイパーパラメータチューニングclass_weight='balanced'を加えて、不均衡データに対応→0.5から0.56になった！！！！！
svm_sig = SVC(kernel='sigmoid', probability=True, random_state=42,class_weight='balanced')#これが一番！！！
#SVC(kernel='poly', probability=True, random_state=42,class_weight='balanced')0.43エラーも出た
#SVC(kernel='rbf', probability=True, random_state=42,class_weight='balanced')0.56 (最初に試した)
grid_search_sig = GridSearchCV(svm_sig, param_grid_sig, cv=10, scoring='accuracy')
grid_search_sig.fit(X_train_sig_scaler, y_train_sig)
# 最適なハイパーパラメータでモデルを再訓練
best_sig_model = grid_search_sig.best_estimator_
best_sig_model.fit(X_train_sig_scaler, y_train_sig)

# RBFカーネルモデルの評価
y_pred_sig = best_sig_model.predict(X_test_sig_scaler)
accuracy_sig = accuracy_score(y_test_sig, y_pred_sig)

# テストデータでの予測確率を取得
# `predict_proba`を使用してクラス1（positiveクラス）の確率を取得
y_scores_sig = best_sig_model.predict_proba(X_test_sig_scaler)[:, 1]

# ROC計算用のしきい値を決定（スコアの一意な値を使用）
thresholds_sig= np.sort(np.unique(y_scores_sig))

# 感度（TPR）と特異度を計算
tpr_list_sig = []
specificity_list_sig = []

for threshold in thresholds_sig:
    # スコアをしきい値で2値化
    y_pred_sig = (y_scores_sig >= threshold).astype(int)
    
    # 混同行列を取得
    tn_sig, fp_sig, fn_sig, tp_sig = confusion_matrix(y_test_sig, y_pred_sig).ravel()
    
    # 感度と特異度を計算
    tpr_sig = tp_sig / (tp_sig+ fn_sig) if (tp_sig + fn_sig) > 0 else 0  # 感度
    specificity_sig = tn_sig / (tn_sig + fp_sig) if (tn_sig + fp_sig) > 0 else 0  # 特異度
    
    # リストに保存
    tpr_list_sig.append(tpr_sig)
    specificity_list_sig.append(specificity_sig)

# AUCの計算
roc_auc_sig = auc(np.array(specificity_list_sig), tpr_list_sig)
# ROC曲線を描画
plt.figure(figsize=(8, 6))
plt.plot( np.array(specificity_list_sig), tpr_list_sig, label=f"ROC Curve (AUC = {roc_auc_sig:.2f})", color="blue")

plt.plot([1, 0], [0, 1], color="gray", linestyle="--")  # 45度線
plt.xlim([1.0, 0.0])  # X軸を1から0に設定
plt.xlabel("Specificity ")
plt.ylabel("Sensitivity")
plt.title("ROC Curve (Specificity vs Sensitivity)")
plt.legend(loc="lower left")
plt.grid()
plt.show()


# RBFカーネルSVMの結果表示
print("最適化されたsigmoidカーネルSVMの精度:", accuracy_sig)
print("sigmoidカーネルSVM最適パラメータ:", grid_search_sig.best_params_)
print("sigmoidカーネルSVMの詳細な評価結果:\n", classification_report(y_test_sig, y_pred_sig))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, confusion_matrix
import numpy as np

# 各モデルから得られた予測スコア（仮定）
y_scores_LR = model_LR.predict_proba(X_test_standard_LR)[:, 1]
y_scores_rf = model_rf.predict_proba(X_test_rf)[:, 1]
y_scores_xgb = model_xgb.predict(dtest_xgb)
y_scores_nn = best_model_nn.predict_proba(X_test_scaled_df_nn)[:, 1]
y_scores_senkei = best_model_senkei.predict_proba(X_test_senkei)[:, 1]
y_scores_sig = best_sig_model.predict_proba(X_test_sig_scaler)[:, 1]

# しきい値を決定（予測スコアの一意な値）
thresholds_LR = np.sort(np.unique(y_scores_LR))
thresholds_rf = np.sort(np.unique(y_scores_rf))
thresholds_xgb = np.sort(np.unique(y_scores_xgb))
thresholds_nn = np.sort(np.unique(y_scores_nn))
thresholds_senkei = np.sort(np.unique(y_scores_senkei))
thresholds_sig = np.sort(np.unique(y_scores_sig))

# 感度と特異度のリストを格納
tpr_list_LR, specificity_list_LR = [], []
tpr_list_rf, specificity_list_rf = [], []
tpr_list_xgb, specificity_list_xgb = [], []
tpr_list_nn, specificity_list_nn = [], []
tpr_list_senkei, specificity_list_senkei = [], []
tpr_list_sig, specificity_list_sig = [], []

# 各モデルにおけるしきい値に対する感度と特異度を計算
for threshold in thresholds_LR:
    y_pred_LR = (y_scores_LR >= threshold).astype(int)
    tn_LR, fp_LR, fn_LR, tp_LR = confusion_matrix(y_test_LR, y_pred_LR).ravel()
    tpr_LR = tp_LR / (tp_LR + fn_LR) if (tp_LR + fn_LR) > 0 else 0  # 感度
    specificity_LR = tn_LR / (tn_LR + fp_LR) if (tn_LR + fp_LR) > 0 else 0  # 特異度
    tpr_list_LR.append(tpr_LR)
    specificity_list_LR.append(specificity_LR)

for threshold in thresholds_rf:
    y_pred_rf = (y_scores_rf >= threshold).astype(int)
    tn_rf, fp_rf, fn_rf, tp_rf = confusion_matrix(y_test_rf, y_pred_rf).ravel()
    tpr_rf = tp_rf / (tp_rf + fn_rf) if (tp_rf + fn_rf) > 0 else 0  # 感度
    specificity_rf = tn_rf / (tn_rf + fp_rf) if (tn_rf + fp_rf) > 0 else 0  # 特異度
    tpr_list_rf.append(tpr_rf)
    specificity_list_rf.append(specificity_rf)

for threshold in thresholds_xgb:
    y_pred_xgb = (y_scores_xgb >= threshold).astype(int)
    tn_xgb, fp_xgb, fn_xgb, tp_xgb = confusion_matrix(y_test_xgb, y_pred_xgb).ravel()
    tpr_xgb = tp_xgb / (tp_xgb + fn_xgb) if (tp_xgb + fn_xgb) > 0 else 0  # 感度
    specificity_xgb = tn_xgb / (tn_xgb + fp_xgb) if (tn_xgb + fp_xgb) > 0 else 0  # 特異度
    tpr_list_xgb.append(tpr_xgb)
    specificity_list_xgb.append(specificity_xgb)

for threshold in thresholds_nn:
    y_pred_nn = (y_scores_nn >= threshold).astype(int)
    tn_nn, fp_nn, fn_nn, tp_nn = confusion_matrix(y_test_nn, y_pred_nn).ravel()
    tpr_nn = tp_nn / (tp_nn + fn_nn) if (tp_nn + fn_nn) > 0 else 0  # 感度
    specificity_nn = tn_nn / (tn_nn + fp_nn) if (tn_nn + fp_nn) > 0 else 0  # 特異度
    tpr_list_nn.append(tpr_nn)
    specificity_list_nn.append(specificity_nn)

for threshold in thresholds_senkei:
    y_pred_senkei = (y_scores_senkei >= threshold).astype(int)
    tn_senkei, fp_senkei, fn_senkei, tp_senkei = confusion_matrix(y_test_senkei, y_pred_senkei).ravel()
    tpr_senkei = tp_senkei / (tp_senkei + fn_senkei) if (tp_senkei + fn_senkei) > 0 else 0  # 感度
    specificity_senkei = tn_senkei / (tn_senkei + fp_senkei) if (tn_senkei + fp_senkei) > 0 else 0  # 特異度
    tpr_list_senkei.append(tpr_senkei)
    specificity_list_senkei.append(specificity_senkei)

for threshold in thresholds_sig:
    y_pred_sig = (y_scores_sig >= threshold).astype(int)
    tn_sig, fp_sig, fn_sig, tp_sig = confusion_matrix(y_test_sig, y_pred_sig).ravel()
    tpr_sig = tp_sig / (tp_sig + fn_sig) if (tp_sig + fn_sig) > 0 else 0  # 感度
    specificity_sig = tn_sig / (tn_sig + fp_sig) if (tn_sig + fp_sig) > 0 else 0  # 特異度
    tpr_list_sig.append(tpr_sig)
    specificity_list_sig.append(specificity_sig)

# AUCの計算
roc_auc_LR = auc(np.array(specificity_list_LR), np.array(tpr_list_LR))
roc_auc_rf = auc(np.array(specificity_list_rf), np.array(tpr_list_rf))
roc_auc_xgb = auc(np.array(specificity_list_xgb), np.array(tpr_list_xgb))
roc_auc_nn = auc(np.array(specificity_list_nn), np.array(tpr_list_nn))
roc_auc_senkei = auc(np.array(specificity_list_senkei), np.array(tpr_list_senkei))
roc_auc_sig = auc(np.array(specificity_list_sig), np.array(tpr_list_sig))

plt.rcParams['font.family'] = 'MS Gothic'  # 日本語対応フォントを指定
# ROC曲線を描画
plt.figure(figsize=(8, 6))

# 各モデルのROC曲線を描画
plt.plot(specificity_list_LR, tpr_list_LR, label=f"ロジスティック回帰 (AUC = {roc_auc_LR:.2f})", color="blue",lw=3,linestyle='-')
plt.plot(specificity_list_rf, tpr_list_rf, label=f"ランダムフォレスト (AUC = {roc_auc_rf:.2f})", color="green",lw=3,linestyle='-')
plt.plot(specificity_list_xgb, tpr_list_xgb, label=f"XGBoost (AUC = {roc_auc_xgb:.2f})", color="red",lw=3,linestyle='-')
plt.plot(specificity_list_nn, tpr_list_nn, label=f"ニューラルネットワーク (AUC = {roc_auc_nn:.2f})", color="purple",lw=3,linestyle='-')
plt.plot(specificity_list_senkei, tpr_list_senkei, label=f"線形サポートベクターマシン (AUC = {roc_auc_senkei:.2f})", color='#3C3C3C',lw=3,linestyle='--')
plt.plot(specificity_list_sig, tpr_list_sig, label=f"非線形サポートベクターマシン (AUC = {roc_auc_sig:.2f})", color='#7D7D7D',lw=3,linestyle='--')

# 45度線を描画
plt.plot([1, 0], [0, 1], color="gray", linestyle="--")

# 軸の設定
plt.xlim([1.0, 0.0])  # X軸を1から0に設定
plt.xlabel('特異度',fontsize=24)
plt.ylabel('感度',fontsize=24)
plt.title('ROC曲線（訓練テスト=8:2）',fontsize=26)
plt.legend(loc='lower right',title="Models",fontsize=13)

# 目盛りのフォントサイズ
plt.xticks(fontsize=18)  # X軸の目盛り
plt.yticks(fontsize=18)  # Y軸の目盛り
# グリッドを表示
plt.grid()

# グラフを表示
plt.show()
